In [ ]:
    !pip install datasetsf

In [ ]:
    from datasets import load_dataset

    # Load a public dataset
    dataset = load_dataset("ainlpml/english-hindi")

    # Or, if you need to load a specific configuration or split
    # dataset = load_dataset("glue", "mrpc")
    # dataset = load_dataset("your_username/your_dataset_name", split="train")

eng.txt:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

hin.txt:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
!pip install huggingface_hub

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `TRECAG` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-creden

In [ ]:
!pip install -q datasets pandas openpyxl


In [ ]:
!pip install -q datasets pandas gspread gspread-dataframe google-auth openpyxl


In [ ]:
# 0) Install required packages
!pip install -q pandas gspread gspread-dataframe google-auth openpyxl

# 1) Download eng.txt and hin.txt from the Hugging Face dataset repo and load into DataFrame
import os
import requests
import pandas as pd
import re
from pathlib import Path

BASE_RAW = "https://huggingface.co/datasets/ainlpml/english-hindi/resolve/main"
ENG_URL = f"{BASE_RAW}/eng.txt"
HIN_URL = f"{BASE_RAW}/hin.txt"

download_dir = "https://huggingface.co/datasets/ainlpml/english-hindi/tree/main"
Path(download_dir).mkdir(parents=True, exist_ok=True)
eng_path = os.path.join(download_dir, "eng.txt")
hin_path = os.path.join(download_dir, "hin.txt")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
eng_path = "/content/drive/MyDrive/AINLP/eng.txt"
hin_path = "/content/drive/MyDrive/AINLP/hin.txt"

# Function to read lines
def read_lines_strip(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

eng_lines = read_lines_strip(eng_path)
hin_lines = read_lines_strip(hin_path)

print("English lines:", len(eng_lines))
print("Hindi lines:", len(hin_lines))



English lines: 10000
Hindi lines: 10000


In [ ]:
!pip install -q pandas openpyxl gspread gspread-dataframe google-auth

import os, re, pandas as pd
from pathlib import Path

# ------------ Edit these if your paths differ ------------
eng_path = "/content/drive/MyDrive/AINLP/eng.txt"
hin_path = "/content/drive/MyDrive/AINLP/hin.txt"
# ------------ end paths ----------------------------------

# Output paths
out_folder = "/content/drive/MyDrive/AINLP"   # saves Excel to this folder in your Drive
Path(out_folder).mkdir(parents=True, exist_ok=True)
excel_out = os.path.join(out_folder, "english_hindi_cleaned.xlsx")
csv_out = os.path.join(out_folder, "english_hindi_cleaned.csv")

# Safety: check files exist
if not os.path.exists(eng_path):
    raise FileNotFoundError(f"English file not found: {eng_path}")
if not os.path.exists(hin_path):
    raise FileNotFoundError(f"Hindi file not found: {hin_path}")

# Read files (strip trailing newlines)
def read_lines_strip(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.rstrip("\n\r") for line in f]

eng_lines = read_lines_strip(eng_path)
hin_lines = read_lines_strip(hin_path)

print("Lines loaded -> English:", len(eng_lines), "Hindi:", len(hin_lines))

Lines loaded -> English: 10000 Hindi: 10000


In [ ]:
# Align lengths (truncate to min length to get pairs)
min_len = min(len(eng_lines), len(hin_lines))
if len(eng_lines) != len(hin_lines):
    print(f"WARNING: line count mismatch — truncating to min length = {min_len}")
eng_lines = eng_lines[:min_len]
hin_lines = hin_lines[:min_len]

# Build DataFrame
df = pd.DataFrame({"English Sentences": eng_lines, "Hindi Sentences": hin_lines})

# Word count function (whitespace tokens)
def word_count_simple(text):
    if text is None: return 0
    s = str(text).strip()
    if s == "": return 0
    return len(re.findall(r'\S+', s))

# Compute word counts
df["Word Count (English)"] = df["English Sentences"].apply(word_count_simple)
df["Word Count (Hindi)"] = df["Hindi Sentences"].apply(word_count_simple)


In [ ]:
# Filter 1: keep only pairs where both counts between 5 and 50 inclusive
min_wc, max_wc = 5, 50
mask_wc = df["Word Count (English)"].between(min_wc, max_wc) & df["Word Count (Hindi)"].between(min_wc, max_wc)
df_after_wc = df[mask_wc].copy()
print("Rows after 5<=word_count<=50 filter (both languages):", len(df_after_wc), " / ", len(df))

# Word count difference and filter 2
df_after_wc["Difference between Word Count (English) and Word Count (Hindi)"] = (
    df_after_wc["Word Count (English)"] - df_after_wc["Word Count (Hindi)"]
)
mask_diff = df_after_wc["Difference between Word Count (English) and Word Count (Hindi)"].between(-10, 10)
df_final = df_after_wc[mask_diff].copy()
print("Rows after diff in [-10,10]:", len(df_final))

# Reorder columns to the requested order
final_cols = [
    "English Sentences",
    "Hindi Sentences",
    "Word Count (English)",
    "Word Count (Hindi)",
    "Difference between Word Count (English) and Word Count (Hindi)"
]
df_final = df_final[final_cols].reset_index(drop=True)

# Save Excel and CSV to your Drive
df_final.to_excel(excel_out, index=False)
df_final.to_csv(csv_out, index=False, encoding="utf-8")
print("Saved final Excel to:", excel_out)
print("Saved final CSV to:", csv_out)

Rows after 5<=word_count<=50 filter (both languages): 8788  /  10000
Rows after diff in [-10,10]: 8214
Saved final Excel to: /content/drive/MyDrive/AINLP/english_hindi_cleaned.xlsx
Saved final CSV to: /content/drive/MyDrive/AINLP/english_hindi_cleaned.csv


In [ ]:
# Show diagnostics & sample
print("\nDiagnostics:")
print("  Original pairs (after align):", min_len)
print("  After 5-50 wc filter:", len(df_after_wc))
print("  After diff filter (-10..10):", len(df_final))
if len(df_final) < 10000:
    print("WARNING: Final cleaned dataset has fewer than 10,000 rows (rows = {}).".format(len(df_final)))
else:
    print("OK: Final cleaned dataset has at least 10,000 rows (rows = {}).".format(len(df_final)))

print("\nSample rows:")
display(df_final.sample(min(8, len(df_final))))

# Optional: upload to Google Sheets (set to True to run)
UPLOAD_TO_SHEETS = True

if UPLOAD_TO_SHEETS:
    try:
        from google.colab import auth
        auth.authenticate_user()  # will prompt to allow access
        import gspread
        from google.auth import default
        from gspread_dataframe import set_with_dataframe

        creds, _ = default(scopes=["https://www.googleapis.com/auth/drive",
                                   "https://www.googleapis.com/auth/spreadsheets"])
        gc = gspread.authorize(creds)
        sheet_title = "English_Hindi_Cleaned"
        sh = gc.create(sheet_title)
        ws = sh.sheet1

        # Write only English (A) and Hindi (B) and the extra columns (you can remove extra columns if you want)
        set_with_dataframe(ws, df_final)
        print("Google Sheet created at:", sh.url)
    except Exception as e:
        print("Google Sheets upload failed or was skipped. Error:", str(e))
        print("You can upload the saved CSV manually into Google Sheets as a fallback.")

# Done
print("\nAll done. If you want a different output filename, a stricter/looser filter, or if you want only A/B columns in the sheet, tell me and I'll modify the cell.")



Diagnostics:
  Original pairs (after align): 10000
  After 5-50 wc filter: 8788
  After diff filter (-10..10): 8214

Sample rows:


,English Sentences,Hindi Sentences,Word Count (English),Word Count (Hindi),Difference between Word Count (English) and Word Count (Hindi)
725,He demanded an inquiry into the case.,उन्होंने इस मामले में जांच की मांग की।,7,8,-1
4153,'Mahagatbandhan' (grand alliance) leaders like...,"महागठबंधन के नेता अखिलेश यादव, मायावती और अजीत...",24,21,3
2873,"""""""The alleged behaviour of these five individ...","इसके अलावा, होमलैंड सिक्यॉरिटी इन्वेस्टिगेशंस ...",28,32,-4
1166,A bench of Chief Justice D N Patel and Justice...,"रिपोर्ट्स के मुताबिक, चीफ जस्टिस डी एन पटेल और...",33,28,5
2074,Smriti Iranis fake degree case: Delhi court ta...,फेक डिग्री विवाद: स्मृति ईरानी के इस्तीफे की म...,13,15,-2
7476,"If some such legislation, as is proposed in Bo...","अगर कोई ऐसा कानून होता, जैसा कि बंबई सरकार बना...",50,49,1
5255,"""""""There are only two lasting gift we should g...",“हम अपने बच्चों को दो ही स्थायी उत्तरदान देने ...,19,25,-6
7496,A heavy police posse was deployed at the spot ...,मौके पर किसी अप्रिय घटना से निपटने के लिए भारी...,14,16,-2


Google Sheet created at: https://docs.google.com/spreadsheets/d/15ZatMGrIr4YmAYyzKD_IU0Q2BuWHrhF8VQp0hkmg8kw

All done. If you want a different output filename, a stricter/looser filter, or if you want only A/B columns in the sheet, tell me and I'll modify the cell.


In [ ]:
!pip install -q transformers sentencepiece sacrebleu pandas openpyxl torch --upgrade

import os, random
import pandas as pd
from pathlib import Path

# ------------- USER CONFIG: modify if needed ----------------
# Path to your cleaned CSV from Assignment 1 (change if different)
INPUT_CSV = "/content/drive/MyDrive/AINLP/english_hindi_cleaned.csv"

# Output files (saved to Drive)
OUTPUT_EXCEL = "/content/drive/MyDrive/AINLP/assessment2_translations_opusmt.xlsx"
METRICS_TXT = "/content/drive/MyDrive/AINLP/assessment2_metrics_opusmt.txt"

# Selection method: "first", "random", or "indices"
SELECT_METHOD = "first"
NUM_SENTENCES = 100
RANDOM_SEED = 42
INDICES = list(range(0,100))  # used if SELECT_METHOD == "indices"
# ----------------------------------------------------------------

# 1) Load input CSV
if not os.path.exists(INPUT_CSV):
    raise FileNotFoundError(f"Input CSV not found: {INPUT_CSV}\nMake sure you mounted Drive and the path is correct.")
df_all = pd.read_csv(INPUT_CSV, encoding="utf-8")
print("Loaded CSV. Columns:", df_all.columns.tolist())


Loaded CSV. Columns: ['English Sentences', 'Hindi Sentences', 'Word Count (English)', 'Word Count (Hindi)', 'Difference between Word Count (English) and Word Count (Hindi)']


In [ ]:
# Detect English/Hindi columns heuristically
possible_eng_cols = [c for c in df_all.columns if "eng" in c.lower() or "english" in c.lower()]
possible_hin_cols = [c for c in df_all.columns if "hin" in c.lower() or "hindi" in c.lower()]

if not possible_eng_cols or not possible_hin_cols:
    raise ValueError("Couldn't auto-detect English/Hindi columns. Please ensure CSV has column names containing 'english' and 'hindi' (case-insensitive).")

eng_col = possible_eng_cols[0]
hin_col = possible_hin_cols[0]
print("Using columns -> English:", eng_col, ", Hindi (reference):", hin_col)


Using columns -> English: English Sentences , Hindi (reference): Hindi Sentences


In [ ]:
# 2) Select 100 sentences
if SELECT_METHOD == "first":
    df_sample = df_all[[eng_col, hin_col]].dropna().head(NUM_SENTENCES).copy()
elif SELECT_METHOD == "random":
    df_sample = df_all[[eng_col, hin_col]].dropna().sample(n=NUM_SENTENCES, random_state=RANDOM_SEED).copy()
elif SELECT_METHOD == "indices":
    df_sample = df_all[[eng_col, hin_col]].dropna().iloc[INDICES].copy()
else:
    raise ValueError("SELECT_METHOD must be one of 'first', 'random', 'indices'")

df_sample = df_sample.reset_index(drop=True)
df_sample.columns = ["english", "hindi_ref"]
print("Selected sentences:", len(df_sample))


Selected sentences: 100


In [ ]:
# 3) Load open-source Hugging Face translation model (opus-mt-en-hi)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

MODEL_NAME = "Helsinki-NLP/opus-mt-en-hi"
print("Loading model:", MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("Model loaded. Using device:", device)


Loading model: Helsinki-NLP/opus-mt-en-hi


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Model loaded. Using device: cpu


In [ ]:
# 4) Translate in batches
def hf_translate(sentences, batch_size=16, max_length=256):
    translations = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        enc = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
        with torch.no_grad():
            out = model.generate(**enc, max_length=max_length, num_beams=4)
        decoded = tokenizer.batch_decode(out, skip_special_tokens=True)
        translations.extend(decoded)
    return translations

src_sentences = df_sample["english"].astype(str).tolist()
print("Translating", len(src_sentences), "sentences...")
translations = hf_translate(src_sentences, batch_size=16)
df_sample["hindi_pred"] = translations


Translating 100 sentences...


In [ ]:
df_sample["hindi_pred"] = translations

In [ ]:
# 5) Evaluate with sacrebleu (corpus-level BLEU, CHRF, TER)
import sacrebleu

references = df_sample["hindi_ref"].astype(str).tolist()
hypotheses = df_sample["hindi_pred"].astype(str).tolist()
refs_for_sacre = [references]  # sacrebleu expects list-of-reference-lists

bleu = sacrebleu.corpus_bleu(hypotheses, refs_for_sacre)
chrf = sacrebleu.corpus_chrf(hypotheses, refs_for_sacre)
ter = sacrebleu.corpus_ter(hypotheses, refs_for_sacre)

print("BLEU:", round(bleu.score,3))
print("CHRF:", round(chrf.score,3))
print("TER :", round(ter.score,3))

BLEU: 15.507
CHRF: 36.291
TER : 83.517


In [ ]:
# 6) Save metrics to TXT
metrics_text = (
    f"Assessment 2 - Translation metrics (open-source model)\n"
    f"Model: {MODEL_NAME}\n"
    f"Selection method: {SELECT_METHOD}, num_sentences: {len(df_sample)}\n\n"
    f"BLEU: {bleu.score:.4f}\n"
    f"CHRF: {chrf.score:.4f}\n"
    f"TER: {ter.score:.4f}\n\n"
    "Notes: BLEU/CHRF/TER computed using sacrebleu (corpus-level).\n"
)
Path(METRICS_TXT).parent.mkdir(parents=True, exist_ok=True)
with open(METRICS_TXT, "w", encoding="utf-8") as f:
    f.write(metrics_text)
print("Saved metrics to:", METRICS_TXT)

Saved metrics to: /content/drive/MyDrive/AINLP/assessment2_metrics_opusmt.txt


In [ ]:
# 7) Save Excel with Original English and Model-generated Hindi
out_df = df_sample[["english", "hindi_pred"]].rename(columns={
    "english": "Original English sentence",
    "hindi_pred": "Model-generated Hindi translation"
})
Path(OUTPUT_EXCEL).parent.mkdir(parents=True, exist_ok=True)
out_df.to_excel(OUTPUT_EXCEL, index=False)
print("Saved Excel to:", OUTPUT_EXCEL)

# 8) Show example rows
print("\nSample translations (reference / prediction):")
display(df_sample[["english","hindi_ref","hindi_pred"]].head(8))

print("\nDone. Files generated:")
print(" - Excel:", OUTPUT_EXCEL)
print(" - Metrics TXT:", METRICS_TXT)

Saved Excel to: /content/drive/MyDrive/AINLP/assessment2_translations_opusmt.xlsx

Sample translations (reference / prediction):


,english,hindi_ref,hindi_pred
0,"However, Paes, who was partnering Australia's ...",आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...,"लेकिन, Poss, जो ऑस्ट्रेलिया के पॉल हैनली के सा..."
1,"Whosoever desires the reward of the world, wit...",और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...,और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...
2,"""The value of insects in the biosphere is enor...","जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ...",जीवविज्ञान में कीड़ों का मूल्य बहुत है क्योंकि...
3,Mithali To Anchor Indian Team Against Australi...,आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को,ओडिय में ऑस्ट्रेलिया के खिलाफ भारतीय टीम के लि...
4,After the assent of the Honble President on 8t...,"8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व...",सन्‌ 1616 में हबल राष्ट्रपति के आने के बाद यान...
5,"""The court has fixed a hearing for February 12",अदालत ने इस मामले में आगे की सुनवाई के लिए एक ...,""" न्यायालय फरवरी 12 के लिए एक सुनवाई तय किया ग..."
6,Please select the position where the track sho...,"जहाँ पर ट्रैक को विभाजित किया जाना है, कृपया व...",कृपया स्थान चुनें जहाँ ट्रैक को विभाजित किया ज...
7,"As per police, armys 22RR, special operation G...",इसके तुरंत बाद सेना की 22 राष्ट्रीय राइफल्स (आ...,"पुलिस की सेना 22R के रूप में, पुलिस समूह के ख़..."



Done. Files generated:
 - Excel: /content/drive/MyDrive/AINLP/assessment2_translations_opusmt.xlsx
 - Metrics TXT: /content/drive/MyDrive/AINLP/assessment2_metrics_opusmt.txt


In [ ]:
# Produce submission Excel with columns:
# Column A: Original English sentence
# Column B: Model-generated Hindi translation
# Paste & run in Colab (assumes Drive mounted if you want to save there).

import os
from pathlib import Path
import glob
import pandas as pd

# ---- USER ADJUSTABLE ----
# final save path (in your Drive)
SAVE_PATH = "/content/drive/MyDrive/AINLP/assessment2_submission.xlsx"
# paths to search for existing translation outputs (in order)
SEARCH_PATHS = [
    "/content",
    "/content/drive/MyDrive/AINLP",
    "/content/drive/MyDrive"
]
# candidate file name patterns to look for if df_sample not present
PATTERNS = ["*assessment2*.xlsx", "*assessment2*.csv", "*translations*.xlsx", "*translations*.csv", "*opusmt*.xlsx", "*opusmt*.csv"]
# --------------------------

def find_files(paths, patterns):
    files = []
    for p in paths:
        for pat in patterns:
            files.extend(glob.glob(os.path.join(p, pat)))
    # remove duplicates and sort
    files = sorted(list(dict.fromkeys(files)))
    return files

# 1) Prefer in-memory df_sample if exists
try:
    df_sample  # noqa: F821
    print("Found in-memory DataFrame 'df_sample'. Using it.")
    df = df_sample.copy()
except NameError:
    # 2) Search disk for likely files
    print("No in-memory 'df_sample' found. Searching disk for likely translation output files...")
    candidates = find_files(SEARCH_PATHS, PATTERNS)
    print("Candidates found:", candidates)
    df = None
    for f in candidates:
        try:
            if f.lower().endswith(".xlsx") or f.lower().endswith(".xls"):
                tmp = pd.read_excel(f, engine="openpyxl")
            else:
                tmp = pd.read_csv(f, encoding="utf-8")
            # check if it contains some useful columns
            cols = [c.lower() for c in tmp.columns]
            if any("english" in c or "original" in c or "source" in c for c in cols) and any("hindi" in c for c in cols):
                df = tmp.copy()
                print("Using file:", f)
                break
        except Exception as e:
            print("Could not read", f, ":", e)
    if df is None:
        raise FileNotFoundError(
            "Could not find an in-memory DataFrame 'df_sample' or any suitable file in SEARCH_PATHS. "
            "Please either (A) make sure df_sample exists in the notebook, or (B) place your translation output file "
            "(CSV/XLSX) in one of the search folders and re-run. Search paths used: " + str(SEARCH_PATHS)
        )

# 3) Normalize column names & pick english and model-generated-hindi columns
cols = df.columns.tolist()
lower_cols = [c.lower() for c in cols]

# Heuristic lists of candidate column names for each role
english_candidates = ["original english sentence","english","english sentence","source","source_text","eng","original english"]
hindi_pred_candidates = ["model-generated hindi translation","hindi_pred","hindi prediction","hindi_predicted","hindi translation","hindi_pred","predicted hindi","hindi_pred","model-generated hindi","hindi_predicted"]

# find best match for english
eng_col = None
for cand in english_candidates:
    for c in cols:
        if cand in c.lower():
            eng_col = c
            break
    if eng_col:
        break
# fallback: any column with 'eng' or 'english' in name
if eng_col is None:
    for c in cols:
        if "eng" in c.lower() or "english" in c.lower():
            eng_col = c
            break

# find best match for predicted hindi
hin_col = None
for cand in hindi_pred_candidates:
    for c in cols:
        if cand in c.lower():
            hin_col = c
            break
    if hin_col:
        break
# fallback: pick a column that contains 'pred' and 'hindi' or 'translation'
if hin_col is None:
    for c in cols:
        if "hindi" in c.lower() and ("pred" in c.lower() or "model" in c.lower() or "translation" in c.lower()):
            hin_col = c
            break

# last resort: if there are exactly 2 columns, assume they are english & hindi (in that order)
if eng_col is None or hin_col is None:
    if len(cols) == 2:
        print("Only two columns found; assuming first is English and second is model Hindi.")
        eng_col, hin_col = cols[0], cols[1]
    else:
        # try pick first column as english and find any other column with 'hindi'
        if eng_col is None:
            for c in cols:
                if "hindi" not in c.lower():
                    eng_col = c
                    break
        if hin_col is None:
            for c in cols:
                if "hindi" in c.lower():
                    hin_col = c
                    break

# Final check
if eng_col is None or hin_col is None:
    raise ValueError(f"Unable to detect English or Model-generated Hindi columns. Available columns: {cols}")

print("Detected columns -> English:", eng_col, "| Model Hindi:", hin_col)

# 4) Build final DataFrame with exact required headers
final_df = pd.DataFrame({
    "Original English sentence": df[eng_col].astype(str).fillna("").tolist(),
    "Model-generated Hindi translation": df[hin_col].astype(str).fillna("").tolist()
})

# If lengths differ (shouldn't), align to min length
min_len = min(len(final_df["Original English sentence"]), len(final_df["Model-generated Hindi translation"]))
final_df = final_df.iloc[:min_len].reset_index(drop=True)

# 5) Save to Excel at SAVE_PATH
out_dir = Path(SAVE_PATH).parent
out_dir.mkdir(parents=True, exist_ok=True)
final_df.to_excel(SAVE_PATH, index=False, engine="openpyxl")
print("Saved submission Excel to:", SAVE_PATH)
print("Rows in submission file:", len(final_df))

# 6) Show a quick sample
display(final_df.head(8))


Found in-memory DataFrame 'df_sample'. Using it.
Detected columns -> English: english | Model Hindi: hindi_pred
Saved submission Excel to: /content/drive/MyDrive/AINLP/assessment2_submission.xlsx
Rows in submission file: 100


,Original English sentence,Model-generated Hindi translation
0,"However, Paes, who was partnering Australia's ...","लेकिन, Poss, जो ऑस्ट्रेलिया के पॉल हैनली के सा..."
1,"Whosoever desires the reward of the world, wit...",और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...
2,"""The value of insects in the biosphere is enor...",जीवविज्ञान में कीड़ों का मूल्य बहुत है क्योंकि...
3,Mithali To Anchor Indian Team Against Australi...,ओडिय में ऑस्ट्रेलिया के खिलाफ भारतीय टीम के लि...
4,After the assent of the Honble President on 8t...,सन्‌ 1616 में हबल राष्ट्रपति के आने के बाद यान...
5,"""The court has fixed a hearing for February 12",""" न्यायालय फरवरी 12 के लिए एक सुनवाई तय किया ग..."
6,Please select the position where the track sho...,कृपया स्थान चुनें जहाँ ट्रैक को विभाजित किया ज...
7,"As per police, armys 22RR, special operation G...","पुलिस की सेना 22R के रूप में, पुलिस समूह के ख़..."
